
# **ANÁLISE DE DADOS PREVENÇÃO DE INADIMPLÊNCIA** 


## Prever e entender o comportamento de clientes inadimplêntes

#### No mundo econômico, a previsibilidade é um fator de extrema importância, já que as tomadas de decisões e investimentos dependem de entender o risco que se corre. Num cenário de concessão de empréstimo, a entidade financeira precisa calcular quais as chances do cliente deixar de efetuar o pagamento, tornando assim inadimplente, prejudicando o investimento.

#### Com a análise de dados dos clientes da entidade financeira, iremos entender quais são as caracteristicas e atitudes que um possível inadimplente possui, levando assim uma melhor tomada de decisão sobre a concessão ou não do crédito.

#### A variável de interesse dos dados é a variável ***default*** que mostra se o cliente está ou já foi inadimplente em algum momento, com os valores booleanos **0 para negativo** e **1 para positivo**.

## **Bibliotecas**

## Pacotes utilizados:

**pandas**: Para a manipulação dos dados

**seaborn** e **matplotlib**: Para visualização gráfica

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## **Dados**

#### Criação do Data Frame utilizando o arquivo CSV anexado.

In [ ]:
df = pd.read_csv('../input/dados-csv-clientes-banco-stone/Python_M10_support material.csv', na_values= 'na')

### Estrutura dos dados

#### Entender a dimensão do Data Frame e analisar a variável de interesse nos diferentes cenários possíveis.

In [ ]:
df.shape
df[df['default'] == 0].shape
df[df['default'] == 1].shape
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape
print(df.shape)
print(df[df['default'] == 0].shape)
print(df[df['default'] == 1].shape)
print(f"A proporção clientes adimplentes é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A proporção clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")

 ####  Nos dados há um total de 10127 clientes e com a variável de interesse *Default* possuindo dois cenários possíveis, **0 para adiplentes** ou **1 para inadimplentes**. Separando a variável de interesse nos dois grupos, obtvemos uma porcentagem de **83.93% de Adiplentes** e **16.07% de Inadimplentes**, essa diferença enorme é natural para uma entidade financeira saudável, onde é interesssante ter mais pagadores fiéis do que devedores.


### Schema

#### Relacionar o assunto de cada coluna com o tipo de dado, visando uma melhor adequação ao problema trabalhado.

#### Colunas e seus respectivos tipos de dados.

In [ ]:
df.dtypes

#### **Atributos Categóricos** (Variáveis explicativas que possuem classificações)


#### Os atributos categóricos são relacionados a categorias, a maioria se encaixa em suas classificações, porém o **"valor_transacoes_12m"** e **"limite_credito"** estão como categóricos e para nosso problema é de interesse de que eles sejam quantificáveis, nescessitando de tratamento desses dados. 

#### Informações estatisticas dos atributos categóricos:

In [ ]:
df.select_dtypes('object').describe().transpose()

#### **Atributos Numéricos** (Variáveis numéricas quantificáveis)

#### Os atributos numéricos também se encaixam na proposta do problema, a variável "ID" nesse Data Frame se encontra como numérico, mas pode ser transformado para categórico caso seja preciso, porém como neste caso não faz sentido utilizarmos será, retirado.

#### Informações estatisticas dos atributos numéricos:

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

#### Os dados se mostram confiáveis e competentes, porém nos atributos categóricos possuem colunas com um ***count*** diferente do total do Data Frame que é de **10127** o que representa que existem dados faltantes no Data Frame e que também precisa ser tratado para melhorar a qualidade das informações e análise.


#### Entender qual o impacto dos dados faltantes nas nossas informações e no balanceamento da variável respsta.

In [ ]:
df.isna().any()

#### Como já era esperado existem 3 colunas com dados faltantes: **escolaridade**, **estado_civil** e **salario_anual**, entender dados estatísticos sobre esses dados faltantes e o impacto nas duas condições da variável resposta ***Default***.

#### Função **stats_dados_faltantes** analisa cada coluna e se encontrar algum dado faltando ela registra a quantidade e gera um dicionário com a porcentagem desse valor em cada coluna. Dando assim a chance de analisar o quanto afeta nosso problema a falta desses dados.


In [ ]:
def stats_dados_faltantes(df: pd.DataFrame) -> None:

  stats_dados_faltantes = []
  for col in df.columns:
    if df[col].isna().any():
      qtd, _ = df[df[col].isna()].shape
      total, _ = df.shape
      dict_dados_faltantes = {col: {'quantidade': qtd, "porcentagem": round(100 * qtd/total, 2)}}
      stats_dados_faltantes.append(dict_dados_faltantes)

  for stat in stats_dados_faltantes:
    print(stat)
stats_dados_faltantes(df=df)

#### *Stats* dados faltantes para Adimplentes:

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 0])

#### *Stats* dados faltantes para inadimplentes:

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 1])

#### Apesar da diferença numérica de dados faltantes entre as duas situações da variável de interesse, nota-se um equilibrio percentual, mostrando que a eliminação dessas linhas pouco afetarão na análise.


## **Transformação e Limpeza dos Dados**

#### Sabendo a estrutura dos dados, vamos adequar eles para melhor atender a análise do nosso problema. Como visto algumas modificações são nescessárias para melhorar a qualidade da informação e nos ajudar nas conclusões.

#### Corrigir o schema das colunas;

#### Remover os dados faltantes.

#### Correção do schema

#### Na etapa de exploração, notamos que as colunas **limite_credito** e **valor_transacoes_12m** estavam sendo interpretadas como colunas categóricas (`dtype = object`) isso acontece por conflito de formato na definição de um dado *float*. E para transformar usaremos uma função para reposicionar as vírgulas e pontos, para que o sistema entenda esse valores como *floats*.

In [ ]:

df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(lambda valor: float(valor.replace(".", "").replace(",", ".")))
df['limite_credito'] = df['limite_credito'].apply(lambda valor: float(valor.replace(".", "").replace(",", ".")))

#### Analisando novamente os tipos de cada coluna e agora já transformadas as colunas **limite_credito** e **valor_transacoes_12m** para atributos numéricos

In [ ]:
df.dtypes

#### Remoção dos dados faltantes

#### Como analisamos anteriormente, as linhas com dados faltantes estão equilibradas proporcionalmente entre as duas condições da variável resposta *Default*, a remoção dessas linhas não afetarão nem a qualidade nem o equilibrio, tornando assim uma melhor visualização. Retornando agora um Data Frame mais enxuto porém com qualidade e equilíbrio dos dados mantidos.

In [ ]:
df.dropna(inplace=True)

df.shape

In [ ]:
qtd_total_novo, _ = df.shape
qtd_adimplentes_novo, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novo, _ = df[df['default'] == 1].shape

print(f"A proporcão adimplentes ativos é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A nova proporcão de clientes adimplentes é de {round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)}%")
print("")
print(f"A proporcão clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")
print(f"A nova proporcão de clientes inadimplentes é de {round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)}%")

#### Conclui-se que não houve perda considerável de dados mas o equilibrio e a qualidade aumentaram, já que agora o Data Frame não possui dados faltantes.

## **Visualização dos Dados**

#### Com os dados trabalhados e lapidados, podemos gerar os gráficos e tabelas que nos ajudaram a entender melhor e prever as características de um possível inadimplente, antes mesmo que ele se torne devedor. 

#### Para melhor compreender a relação entre a variável resposta ***Default*** com as outras variáveis, geramos dois data frames diferentes cada um com um grupo da variável de interesse, um para os **Adimplentes** e outro para os **Inadimplentes**.


In [ ]:
df_adimplente = df[df['default'] == 0]
df_inadimplente = df[df['default'] == 1]

sns.set_style("whitegrid")

#### Visualizar a variável resposta com os atributos categóricos e numéricos separadamente, já que as estatísticas são diferentes em cada modalidade, e sempre fazendo a comparação entre o data frame completo, com os data frames recém criados separando a variável de interesse.

#### Vale a pena deixar claro que essa análise segue as diretrizes estabelecidas pela **LGPD (Lei Geral de Proteção de Dados)**. Não irá ser feito análise com nenhuma variável sensível.



#### **Visualizações Categóricas**

#### **Escolaridade**

In [ ]:
coluna = 'escolaridade'
titulos = ['Escolaridade dos Clientes', 'Escolaridade dos Clientes Adimplentes', 'Escolaridade dos Clientes Inadimplentes']

eixo = 0
max_y = 0
max = df.select_dtypes('object').describe()[coluna]['freq'] * 1.1

figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  df_to_plot = dataframe[coluna].value_counts().to_frame()
  df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace=True)
  df_to_plot[coluna] = df_to_plot.index
  df_to_plot.sort_values(by=[coluna], inplace=True)
  df_to_plot.sort_values(by=[coluna])

  f = sns.barplot(x=df_to_plot[coluna], y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
  f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

#### Nota-se que os clientes com formação superior são a maioria, porém número de clientes sem nenhuma formação também é grande, mostrando um extremo no perfil de escolaridade.

#### É muito interessante entendermos que o objetivo dessa análise é identificar comportamentos e características que os clientes inadimplentes apresentam diferentemente dos adimplentes, como vemos no atributo **"Escolaridade"** o padrão se mantém em níveis muito parecidos nos três gráficos, não se vê de nesse caso, como o grau de escolaridade afeta em o cliente ser ou não inadimplente.

#### **Salário Anual**

In [ ]:
coluna = 'salario_anual'
titulos = ['Salário Anual dos Clientes', 'Salário Anual dos Clientes Adimplentes', 'Salário Anual dos Clientes Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  df_to_plot = dataframe[coluna].value_counts().to_frame()
  df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace=True)
  df_to_plot[coluna] = df_to_plot.index
  df_to_plot.reset_index(inplace=True, drop=True)
  df_to_plot.sort_values(by=[coluna], inplace=True)

  f = sns.barplot(x=df_to_plot[coluna], y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
  f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)
  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))
  eixo += 1

figura.show()

#### No atributo **"Salário Anual"** percebemos que a grande maioria dos clientes da entidade financeira tem um salário anual menor que 40K, informação interessante para tipos de investimentos e perfis de carteiras, porém nosso caso é saber o que interfere no cliente ser um devedor. E não é o que vemos analisando os 3 graficos, já que o padrão também é bem parecido, o comportamento é o mesmo, provando que isso não interfere no nosso problema. 

#### **Estado Civil**

In [ ]:
coluna = 'estado_civil'
titulos = ['Estado Civil dos Clientes', 'Estado Civil dos Clientes Adimplentes', 'Estado Civil dos Clientes Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 20), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  df_to_plot = dataframe[coluna].value_counts().to_frame()
  df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace=True)
  df_to_plot[coluna] = df_to_plot.index
  df_to_plot.reset_index(inplace=True, drop=True)
  df_to_plot.sort_values(by=[coluna], inplace=True)

  f = sns.barplot(x=df_to_plot[coluna], y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
  f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)
  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))
  eixo += 1

figura.show()

#### Estado civil dos clientes é uma informação importante para consceção de seguros e outros pacotes direcionado a família, porém no nosso caso o padrão se mantém o mesmo entre os 3 gráficos mostrando também que o **estado civil não interfere** diretamente em um comportamento inadimplente.

#### Analisando 3 dos principais atributos categóricos, não conseguimos ver padrões diferentes no comportamento dos clientes inadimplentes e adimplentes, podemos concluir que as diferentes categorias que estão destribuídos não tem relação com adimplencia. Vamos analisar agora os fatores quantitativos.

#### **Visualizações Numéricas**

#### **Quantidade de Transações no Último Ano**

In [ ]:
coluna = 'qtd_transacoes_12m'
titulos = ['Qtd. de Transações no Último Ano', 'Qtd. de Transações no Último Ano de Adimplentes', 'Qtd. de Transações no Último Ano de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

#### Para atributos quantitativos o histograma é ideal já que mostra a frquencia de algum valor dentro da amostragem, na variável " qtd_transacoes_12m " temos a relação entre a frequencia de vezes que certa quantidade de transações anuais foram feitas, e fica fácil de visualizar a diferença entre os grupos da variável de interesse.

#### O pico do gráfico dos **adimplentes gira em torno clientes que tem transações anuais em torno de 80 vezes**, já o pico dos **inadimplentes tem transações anuais em torno de 40 vezes**, metade da quantidade de transação dos adimplentes.

#### Nota-se então uma diferença comportamental entre os Adimplentes e Inadimplentes, que é a quantidade anual de transações, **clientes que fazem mais transações durante o ano tendem a ser melhores pagadores que aqueles que fazem poucas transações.**

#### **Valor das Transações nos Últimos 12 Meses**

In [ ]:
coluna = 'valor_transacoes_12m'
titulos = ['Valor das Transações no Último Ano', 'Valor das Transações no Último Ano de Adimplentes', 'Valor das Transações no Último Ano de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

#### Seguindo a mesma lógica da quantidade de transações anuais, vamos analisar o valor total anual das transações, um histograma também mostrando a frequencia de vezes que os dados se repetem em cada situação. E novamente vemos uma diferença no padrão das situações da variável de interesse, o pico do gráfico dos adimplentes se localiza no 5.000,00 enquanto dos inadimplentes se localiza nos 2.500,00 metade do valor dos adimplentes.

#### Por essa visualização podemos concluir que **quanto maior a quantia movimentada por um cliente, menor tbm a chance desse cliente se tornar inadimplente.**

#### **Valor de Transações nos Últimos 12 Meses x Quantidade de Transações nos Últimos 12 Meses**

In [ ]:
f = sns.relplot(x='valor_transacoes_12m', y='qtd_transacoes_12m', data=df, hue='default')
_ = f.set(
    title='Relação entre Valor e Quantidade de Transações no Último Ano', 
    xlabel='Valor das Transações no Último Ano', 
    ylabel='Quantidade das Transações no Último Ano'
  )

#### Usando um gráfico de relação entre a quantidade e valor das transações, fica muito claro que **quanto menos transações e menor o valor transicionado, maior a chance de uma pessoa se tornar inadimplente.**

#### Entendendo isso a entidade financeira pode tentar prever e agir antes em clientes que estão seguindo esse padrão, a fim de gerar soluções economicas para que ele não se torne um cliente inadimplente.  

#### **Limite de crédito**

#### Analisamos também se existe alguma relação entre o valor do crédito disponível, se um valor mais alto de crédito tende a formar mais inadimplentes.



In [ ]:
coluna = 'limite_credito'
titulos = ['Limite de Crédito', 'Limite de Crédito de Adimplentes', 'Limite de Crédito de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

#### Nota-se que o padrão é comum e o limite de crédito tende a influenciar inadimplentes nos dois extremos, limites abaixo de 5000 e maior que 30000 merecem uma atenção, mas não varia de um grupo para o outro, o limite de crédito por sí só não influencia direto na variável de interesse.

## **Conclusões Finais**

#### Nosso problema principal era entender e tentar prever comportamento de clientes que possam se tornar inadimplentes de uma entidade financeira que concede empréstimos, com base na análise de dados do último ano. 

#### Entendemos que a nossa variável de interesse "*Default*", possui dois grupos distintos de adimplentes e de inadimplentes, e o trabalho era de entender a relação da variável de interesse com as demais variáveis preditivas.

#### Depois de trabalhar os dados visando um melhor aproveitamento, foi gerado visualizações comparativas para as principais variáveis relacionando as sempre com a variável de interesse. Como foi observado atributos categóricos como **Salário**, **Escolaridade**, **Estado Civil** pouco diferem no comportamento de adimplentes e inadimplentes.

#### Já os atributos numéricos, **como a quantidade de transações** e o **valor das transações** demonstraram um padrão diferente, e com base nessa análise conclui-se **que quanto menos transações e menor o valor transicionado, maior a chance de uma pessoa se tornar inadimplente.**

#### Conclui-se também que os extremos de **limite de crédito** possuem maiores inadimplentes, tendo atenção para baixos e altos limites.

#### Com essas informações é possivel prever e tratar clientes que possuam características e comportamentos similares, evitando assim que se tornem inadimplentes.